In [39]:
from langchain_community.document_loaders import GitLoader
from langchain_community.vectorstores.hanavector import HanaDB
from hdbcli import dbapi
from langchain_core.embeddings.fake import FakeEmbeddings
from langchain_core.embeddings import Embeddings

In [210]:
#from langchain_core.embeddings import Embeddings
from hdbcli import dbapi

class HanaEmbeddings(Embeddings):
    def __init__(
        self,
        connection: dbapi.Connection
    ):
        self.connection = connection

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return [self.embed_query(text) for text in texts]

    def embed_query(self, text: str) -> list[float]:
        mycur = self.connection.cursor()
        # print(type(text))
        select_sql=f"""SELECT TO_NVARCHAR(VECTOR_EMBEDDING('{text.replace("'","''")}', 'DOCUMENT', 'SAP_NEB.20240715')) FROM DUMMY;"""
        print(select_sql)
        mycur.execute(
            select_sql
        )
        rows = mycur.fetchone()
        return eval(rows[0])

In [214]:
from langchain_core.embeddings import Embeddings
from hdbcli import dbapi

class HanaEmbeddings(Embeddings):
    def __init__(
        self,
        connection: dbapi.Connection
    ):
        self.connection = connection

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return [self.embed_query(text) for text in texts]

    def embed_query(self, text: str) -> list[float]:
        mycur = self.connection.cursor()
        # print(type(text))
        select_sql=f"""SELECT TO_NVARCHAR(VECTOR_EMBEDDING(?, 'DOCUMENT', 'SAP_NEB.20240715')) FROM DUMMY;"""
        # print(select_sql)
        mycur.execute(
            select_sql, text
        )
        rows = mycur.fetchone()
        return eval(rows[0])

In [198]:
hanaemb=HanaEmbeddings(connection_to_hana)

In [199]:
len(hanaemb.embed_query('Hello World!'))

768

In [7]:
loader = GitLoader(
    clone_url="https://github.com/SAP/terraform-provider-btp",
    repo_path="./gen/docs/",
    file_filter=lambda file_path: file_path.startswith("./gen/docs/docs")
    and file_path.endswith(".md"),
    branch="main",
)
documents = loader.load()

len(documents)

81

In [53]:
## Dev Advocates central instance
myhost='c5889dd5-e0f6-4930-8408-94d53ca61dbf.hna0.prod-us10.hanacloud.ondemand.com'
myport=443
myuser='DBADMIN_VR'

In [149]:
connection_to_hana = dbapi.connect(
            address=myhost,
            port=myport,
            user=myuser,
            password=input(),
            encrypt=True,
            sslValidateCertificate=False,
        )

In [215]:
db = HanaDB(
    embedding=HanaEmbeddings(connection_to_hana), connection=connection_to_hana, table_name="MY_LANGCHAIN_EMBEDDINGS"
)

In [218]:
db.delete(filter={})

True

In [219]:
db.add_documents(documents=documents)

[]